# **NUMBER PREDICTION MNIST PYTORCH MODEL**

Author: Doron Levinson

Date: 16.04.2025

### **STEP 1: SET UP & DATA LOADING**

1. Mounting notebook to my personal drive
2. Finding and saving paths to train and test data (data from https://www.kaggle.com/datasets/hojjatk/mnist-dataset)
3. Saving train and test as pandas dataframes using pandas.read_csv

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Finding path of the data
!ls "/content/drive/My Drive/Code Projects/Number Prediction/Data"

# Sabing the paths of train and test data
train_path = "/content/drive/My Drive/Code Projects/Number Prediction/Data/mnist_train.csv"
test_path = "/content/drive/My Drive/Code Projects/Number Prediction/Data/mnist_test.csv"

mnist_test.csv	mnist_train.csv


In [ ]:
# Svaing as pandas data frame using pandas.read_csv() function
import pandas
train_data = pandas.read_csv(train_path)
test_data = pandas.read_csv(test_path)

train_data.head(10)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### **STEP 2: PYTORCH MODEL BUILDING**

1. Importing relevant libraries
2. Data preprocessing, converting to pytorch tensors, and creating DataLoaders
3. Defining a simple feed forward neural network
4. Initializing model, loss function, and optimization method
5. Training Loop
6. Evaluation


In [ ]:
# Loading relevant libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [ ]:
# Spliting train and test data to labels (y) and feature (X)
X_train = train_data.drop(columns='label')
y_train = train_data['label']
X_test = test_data.drop(columns='label')
y_test = test_data['label']

# Normalizing X values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Turning X and y from dataframes to torch tensors
X_train = torch.tensor(X_train.values, dtype = torch.float32)
y_train = torch.tensor(y_train.values, dtype = torch.long)
X_test = torch.tensor(X_test.values, dtype = torch.float32)
y_test = torch.tensor(y_test.values, dtype = torch.long)

# Cfreating DataLoaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size = 64, shuffle = True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size = 64)


In [ ]:
# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
# Initializing model, loss, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Model Training loop
for epoch in range(5):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 314.7039
Epoch 2, Loss: 126.5703
Epoch 3, Loss: 85.4747
Epoch 4, Loss: 64.7861
Epoch 5, Loss: 50.8712


In [ ]:
# Evaluation
model.eval()
correct = total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, preds = torch.max(outputs, 1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 97.73%


### **STEP 3: MODEL SAVING & AWS S3 UPLOADING**

In [ ]:
# Saving model as pth file
torch.save(model.state_dict(), "simple_mnist_model.pth")

In [ ]:
# Install and import boto3
#!pip install boto3
import boto3

In [ ]:
# Set up session using access key and secret key
session = boto3.Session(
    aws_access_key_id='<Add key here>',
    aws_secret_access_key='<Add key here>',
    region_name='<Add region here>'
)

s3 = session.client('s3')

In [ ]:
# Upload the model to the S3 bucket
s3.upload_file("simple_mnist_model.pth", "<bucket name>", "<model_name>.pth")
print("Upload successful!")

Upload successful!
